# Chapter 16. Operator Overloading

Main contents:
 - How an infix operator method should signal it cannot handle an operand
 - Using duck typing or goose typing to deal with operands of various types
 - special behaviour of the rich comparison operators (`==`, `>`, `>=`)
 - default handling of augmented assignment operators such as `+=` and how to overload them

## Operator Overloading 101

restrictions imposed by python
 - we cannot change the meaning of the operators for the built-in types
 - we cannot create new operators, only overload existing ones
 - a few operators can't be overloaded: `is`, `and`, `or`, `not` (but the bitwise can)


## Unary Operator

Note. Stick to the general rule of operators: always return a new object. i.e., don't modify the receiver (`self`), but create and return a new instance of a suitable type

In [1]:
# vector_v5.py
from array import array
import operator
import reprlib
import math
import functools
import itertools

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return(self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)

### When x and +x are not equal

Everybody expects that `x == +x` but there are two cases in the standard library where `x != +x`

In [9]:
import decimal
ctx = decimal.getcontext()
ctx.prec = 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')

In [10]:
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [11]:
one_third == +one_third

False

In [12]:
ctx.prec = 28

In [13]:
one_third == +one_third

False

In [14]:
+one_third

Decimal('0.3333333333333333333333333333')